# Practical 2: Association tests

Objectives:
- Be comfortable with design and control of simple genome-wide association tests.
- Understand basic principles behind simple variant aggregation and burden tests.

In [1]:
import hail as hl
from hail.plot import output_notebook, show

Now we initialize Hail and set up plotting to display inline in the notebook.

In [2]:
hl.init()
# make plots display inline, rather than creating files
output_notebook()

Running on Apache Spark version 2.4.0
SparkUI available at http://10.0.0.74:4043
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.35-577378849928
LOGGING: writing to /Users/kumar/Dropbox (Partners HealthCare)/HailTeam/Workshops/BroadE/hail-20200417-1727-0.2.35-577378849928.log


Loading BokehJS ...

# Part 1: GWAS

## Read QC'ed data
First, we need to read back in the QCed dataset.

In [5]:
!ls -lhat resources/

total 51504
drwx------@ 19 kumar  staff   608B Apr 17 17:27 ..
drwxr-xr-x@ 14 kumar  staff   448B Apr 17 17:06 1kg.mt
drwx------@ 11 kumar  staff   352B Apr 17 17:06 .
-rw-------@  1 kumar  staff    22M Feb 25 12:47 1kg.vcf.bgz
-rw-------@  1 kumar  staff    84K Feb 25 12:47 1kg.fam
-rw-------@  1 kumar  staff   150K Feb 25 12:47 1kg_annotations.txt
-rw-------@  1 kumar  staff   2.5M Feb 25 12:47 ensembl_gene_annotations.txt
-rw-------@  1 kumar  staff    41K Feb 25 12:47 true_pops.txt
drwx------@ 12 kumar  staff   384B Feb 25 12:47 pca_scores.ht
drwx------@ 13 kumar  staff   416B Feb 25 12:47 post_qc.mt
-r--------@  1 kumar  staff     0B Dec 31  1969 Icon?


In [6]:
mt = hl.read_matrix_table('resources/post_qc.mt')

FatalError: JsonMappingException: No content to map due to end-of-input
 at [Source: (is.hail.io.fs.WrappedSeekableDataInputStream); line: 1, column: 0]

Java stack trace:
com.fasterxml.jackson.databind.JsonMappingException: No content to map due to end-of-input
 at [Source: (is.hail.io.fs.WrappedSeekableDataInputStream); line: 1, column: 0]
	at com.fasterxml.jackson.databind.JsonMappingException.from(JsonMappingException.java:148)
	at com.fasterxml.jackson.databind.ObjectReader._initForReading(ObjectReader.java:381)
	at com.fasterxml.jackson.databind.ObjectReader._bindAndClose(ObjectReader.java:1494)
	at com.fasterxml.jackson.databind.ObjectReader.readValue(ObjectReader.java:1102)
	at org.json4s.jackson.JsonMethods$class.parse(JsonMethods.scala:27)
	at org.json4s.jackson.JsonMethods$.parse(JsonMethods.scala:55)
	at is.hail.variant.ReferenceGenome$.read(ReferenceGenome.scala:570)
	at is.hail.variant.ReferenceGenome$$anonfun$readReferences$1$$anonfun$14.apply(ReferenceGenome.scala:657)


Next, we will filter to common variants (those with an alternate allele frequency over 1%). GWAS cannot detect signal from extremely rare variants, like those only observed in one individual.

In [ ]:
mt = mt.filter_rows(hl.agg.call_stats(mt.GT, mt.alleles).AF[1] > 0.01)

In [ ]:
mt.describe(widget=True)

Remember that in a GWAS we're running independent association tests on each variant.

In Hail, the method we use is [hl.linear_regression_rows](https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression_rows). Why isn't this called `hl.gwas`? Modularity!

We use the phenotype `caffeine_consumption` as our dependent variable, the number of alternate alleles as our independent variable, and no covariates besides an intercept term (that's the `1.0`).

In [ ]:
gwas = hl.linear_regression_rows(y=mt.pheno.caffeine_consumption, 
                                 x=mt.GT.n_alt_alleles(), 
                                 covariates=[1.0])

In [ ]:
gwas.describe(widget=True)

## Visualization

Let’s visualize our association test results from the linear regression. We can do so by creating 2 common plots: a [Manhattan plot](https://en.wikipedia.org/wiki/Manhattan_plot) and a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot).

We'll start with the Manhattan plot:

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

The other common plot is the Q-Q (quantile-quantile) plot.

In [ ]:
p = hl.plot.qq(gwas.p_value)
show(p)

## Confounded!

The Q-Q plot indicates **extreme** inflation of p-values.

If you've done a GWAS before, you've probably included a few other covariates that might be confounders -- age, sex, and principal components.

Principal components are a measure of genetic ancestry, and can be used to control for [population stratification](https://en.wikipedia.org/wiki/Population_stratification).

Principal component analysis (PCA) is a very general statistical method for reducing high dimensional data to a small number of dimensions which capture most of the variation in the data. Hail has the function [pca](https://hail.is/docs/0.2/methods/stats.html#hail.methods.pca) for performing generic PCA.

PCA typically works best on normalized data (e.g. mean centered). Hail provides the specialized function [hwe_normalized_pca](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca) which first normalizes the genotypes according to the Hardy-Weinberg Equilibium model.

In [ ]:
pca_eigenvalues, pca_scores, pca_loadings = hl.hwe_normalized_pca(mt.GT, compute_loadings=True)

The pca function returns three things:
* **eigenvalues**: an array of doubles
* **scores**: a table keyed by sample
* **loadings**: a table keyed by variant

The **loadings** are the *principal directions*, each of which is a weighted sum of variants (a "virtual variant"). By default, `hwe_normalized_pca` gives us 10 principal directions.

In [ ]:
pca_loadings.show()

The **scores** are the principal components themselves, computed per sample. The scores are the projection of each sample onto the principal directions. We can think of a sample's scores as its "virtual genome", which is now just 10 numbers.

In [ ]:
pca_scores.show()

The **eigenvalues** reflect the relative amount of variance explained by each principal component:

In [ ]:
pca_eigenvalues

We can **annotate** the principal components back onto `mt` (recall annotating population/phenotype labels in the last practical):

In [ ]:
mt = mt.annotate_cols(pca = pca_scores[mt.s])

## Control confounders and run another GWAS

Now that we have computed principal components and saved it into our `mt`, let’s attempt to adjust the inflation that we saw in our initial Q-Q plot. We will now add PCs as `covariates` in `linear_regression_rows`:


In [ ]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.caffeine_consumption, 
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.is_female, mt.pca.scores[0], mt.pca.scores[1], mt.pca.scores[2]])

In [ ]:
p = hl.plot.qq(gwas.p_value)
show(p)

The above Q-Q plot indicates much better (but not good!) genomic control. Let's try the Manhattan plot:

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

# Part 2: Rare variant grouping and burden

GWAS is a great tool for finding associations between **common variants** and disease, but is underpowered to detect rare-variant associations, because rare variants by definition have small sample sizes.

It is possible to find associations between rare variants and disease by **grouping variants of similar effect**, and testing each group.

One possible solution is to sum variant counts according to some genomic interval (for instance, gene), and then association with these intervals. A version of this kind of test is called a burden test. 

We'll do a burden test that associates rare variant burden with our `caffeine_consumption` phenotype. We shouldn't hope to find anything here -- especially because we've only got a few thousand rare variants!


# <font color="#1a0dab">Step 1:</font> Import variant data

First, we'll need to start again from the QC'ed matrix table on disk -- `mt` has been filtered to include only common variants.

In [ ]:
mt = hl.read_matrix_table('resources/post_qc.mt')

Next, we will remove variants with allele frequency under 1%. Including common variants will only reduce the power of a burden test.

In [ ]:
mt = mt.filter_rows(hl.agg.call_stats(mt.GT, mt.alleles).AF[1] < 0.01)

# <font color="#1a0dab">Step 2:</font> Group by gene


To assign variants to genes, we'll use a tab-separated file that contains genomic intervals and corresponding genes.


In [ ]:
gene_ht = hl.import_table('resources/ensembl_gene_annotations.txt', impute=True)

In [ ]:
gene_ht.show()

How many intervals (genes) are there?

In [ ]:
gene_ht.count()

## Annotate variants with genes

In order join our two tables, we need to create a field of type `interval` so that Hail knows how to execute a join.

We'll use the [transmute](https://hail.is/docs/0.2/hail.Table.html?highlight=transmute#hail.Table.transmute) function, which is like `annotate`, but drops any fields referenced in the computation.

In [ ]:
print('before transmute')
gene_ht.describe()

gene_ht = gene_ht.transmute(
    interval = hl.locus_interval(gene_ht.chromosome,
                                 gene_ht.start,
                                 gene_ht.end))

print('')
print('after transmute')
gene_ht.describe()

This field needs to be the key of the table, so we will use [key_by](https://hail.is/docs/0.2/hail.Table.html?highlight=key_by#hail.Table.key_by) to assign this computed field as the table key:

In [ ]:
keyed_gene_table = gene_ht.key_by('interval')

keyed_gene_table.describe()

Recall how we annotated sample phenotypes earlier -- this join looks very similar:

In [ ]:
mt = mt.annotate_rows(gene = keyed_gene_table[mt.locus].gene_name)

Let's `show` the resulting annotations on the matrix table. How do they differ?

In [ ]:
mt.gene.show()

## Step 3: Aggregate by gene

Hail's modularity makes it easy to perform non-kernel-based burden tests.

We'll compose two general tools:
 - [group_rows_by](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.group_rows_by) / [aggregate](https://hail.is/docs/0.2/hail.GroupedMatrixTable.html#hail.GroupedMatrixTable.aggregate)
 - [hl.linear_regression_rows](https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression_rows).
 
This means that you can flexibly specify the way genotypes are summarized per gene. Using other tools, you may have a few ways to aggregate, but if you want to do something different you are out of luck!

In [ ]:
mt.describe(widget=True)

In [ ]:
burden_mt = (
    mt
    .group_rows_by('gene')
    .aggregate(n_variants = hl.agg.count_where(mt.GT.n_alt_alleles() > 0))
)

# filter to genes with at least one rare variant!
burden_mt = burden_mt.filter_rows(hl.agg.sum(burden_mt.n_variants) > 0)

In [ ]:
burden_mt.describe(widget=True)

In [ ]:
burden_mt.show()

## Step 4: Run linear regression per gene

This should look familiar! We can reuse the same modular components (like `linear_regression_rows`) for many different purposes.

In [ ]:
burden_mt = burden_mt.annotate_cols(pca = pca_scores[burden_mt.s])

burden_results = hl.linear_regression_rows(
    y=burden_mt.pheno.caffeine_consumption, 
    x=burden_mt.n_variants,
    covariates=[1.0, 
                burden_mt.pheno.is_female, 
                burden_mt.pca.scores[0], 
                burden_mt.pca.scores[1], 
                burden_mt.pca.scores[2]])

## Sorry, no `hl.plot.manhattan` for genes!

Manhattan plots are really only useful for standard GWAS. Instead, we can simply sort by p-value using [order_by](https://hail.is/docs/0.2/hail.Table.html#hail.Table.order_by), and print:

In [ ]:
burden_results.order_by(burden_results.p_value).show()